In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from icecream import ic
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

SubmitId = test_data['Id']

# 去掉异常值 2个
train_data.drop(train_data[(train_data['GrLivArea'] > 4000) & (train_data['SalePrice'] < 300000)].index, inplace=True)

train_data['SalePrice_Log'] = np.log(train_data['SalePrice'])

# 合并训练数据和测试数据
all_data = pd.concat([train_data, test_data], ignore_index=True)

# 填补 MSZoning 缺失值
a = all_data.groupby(by=['MSSubClass', 'Street'])['MSZoning'].value_counts().reset_index(name='count')
all_data.loc[all_data.MSZoning.isnull(), 'MSZoning'] = all_data.loc[all_data.MSZoning.isnull(), :].apply(lambda x: a.loc[(x.MSSubClass == a.MSSubClass) & (x.Street == a.Street), 'MSZoning'].values[0], axis=1)

# 填补 Utilities 缺失值
a = all_data.groupby(by=['MSSubClass'])['Utilities'].value_counts().reset_index(name='count')
all_data.loc[all_data.Utilities.isnull(), 'Utilities'] = all_data.loc[all_data.Utilities.isnull()].apply(lambda x: a.loc[(x.MSSubClass == a.MSSubClass), 'Utilities'].values[0], axis=1)

# 均值法填补 LogFrontage [MSSubClass,OverallCond,BldgType,LotConfig,Neighborhood,MSZoning]
a = all_data[all_data.LotFrontage.notnull()].groupby(by=['MSSubClass', 'OverallCond', 'BldgType'])['LotFrontage'].mean()
a = a.reset_index()[['MSSubClass', 'OverallCond', 'BldgType', 'LotFrontage']]
def fill_lf(row):
    fill_val = a[(a.MSSubClass == row.MSSubClass) & (a.OverallCond == row.OverallCond) & (a.BldgType == row.BldgType)]['LotFrontage'].values
    if len(fill_val) == 0:
        return np.nan
    else:
        return fill_val[0]
all_data.loc[all_data.LotFrontage.isnull(), 'LotFrontage'] = all_data[all_data.LotFrontage.isnull()].apply(fill_lf, axis=1)

"""第二次"""
a = all_data[all_data.LotFrontage.notnull()].groupby(by=['MSSubClass'])['LotFrontage'].mean()
a = a.reset_index()[['MSSubClass', 'LotFrontage']]

def fill_lf2(row):
    fill_val = a[(a.MSSubClass == row.MSSubClass)]['LotFrontage'].values
    if len(fill_val) == 0:
        return np.nan
    else:
        return fill_val[0]
all_data.loc[all_data.LotFrontage.isnull(), 'LotFrontage'] = all_data[all_data.LotFrontage.isnull()].apply(fill_lf2, axis=1)

"""第三次"""
all_data.loc[all_data.LotFrontage.isnull(), 'LotFrontage'] = all_data.loc[all_data.LotFrontage.notnull(), 'LotFrontage'].mean()

# 采用众数法填补Exterior1st
all_data.loc[all_data.Exterior1st.isnull(), 'Exterior1st'] = 'VinylSd'

# 采用众数法填补Exterior1st
all_data.loc[all_data.Exterior2nd.isnull(), 'Exterior2nd'] = 'VinylSd'

# 填补 MasVnrType 缺失值
a = all_data.groupby(by=['BldgType', 'HouseStyle', 'OverallQual', 'MSSubClass'])['MasVnrType'].value_counts().reset_index(name='count')
all_data.loc[all_data.MasVnrType.isnull(), 'MasVnrType'] = all_data.loc[all_data.MasVnrType.isnull(), :].apply(lambda x: a.loc[(x.BldgType == a.BldgType) & (x.HouseStyle == a.HouseStyle) & (x.OverallQual == a.OverallQual) & (x.MSSubClass == a.MSSubClass), 'MasVnrType'].values[0], axis=1)

# 填补 MasVnrArea 缺失值
"""第一次"""
all_data.loc[all_data.MasVnrType == 'None', ['MasVnrArea']] = 0.0

"""第二次"""
a = all_data[(all_data.MasVnrArea.notnull()) & (all_data.MasVnrType != 'None')].groupby(by=['OverallQual'])['MasVnrArea'].mean().reset_index()[['OverallQual', 'MasVnrArea']]
def fill_mvr(row):
    t = a.loc[(a.OverallQual == row.OverallQual), 'MasVnrArea']
    return t.values[0]
all_data.loc[all_data.MasVnrArea.isnull(), 'MasVnrArea'] = all_data[all_data.MasVnrArea.isnull()].apply(fill_mvr, axis=1)

# 填补 BsmtQual 缺失值
def fill_BsmtQual(row):
    t = all_data.loc[(all_data.OverallCond == row.OverallCond), 'BsmtQual'].value_counts()
    return t.index[0]
all_data.loc[all_data.BsmtQual.isnull(), 'BsmtQual'] = all_data[all_data.BsmtQual.isnull()].apply(fill_BsmtQual, axis=1)

# 填补 BsmtCond 缺失值
def fill_BsmtCond(row):
    t = all_data.loc[(all_data.OverallCond == row.OverallCond), 'BsmtCond'].value_counts()
    return t.index[0]
all_data.loc[all_data.BsmtCond.isnull(), 'BsmtCond'] = all_data[all_data.BsmtCond.isnull()].apply(fill_BsmtCond, axis=1)

# 填补 BsmtExposure 缺失值
def fill_BsmtExposure(row):
    t = all_data.loc[(all_data.OverallCond == row.OverallCond), 'BsmtExposure'].value_counts()
    return t.index[0]
all_data.loc[all_data.BsmtExposure.isnull(), 'BsmtExposure'] = all_data[all_data.BsmtExposure.isnull()].apply(fill_BsmtExposure, axis=1)

# 填补 BsmtFinType1 缺失值
def fill_BsmtFinType1(row):
    t = all_data.loc[(all_data.OverallCond == row.OverallCond), 'BsmtFinType1'].value_counts()
    return t.index[0]
all_data.loc[all_data.BsmtFinType1.isnull(), 'BsmtFinType1'] = all_data[all_data.BsmtFinType1.isnull()].apply(fill_BsmtFinType1, axis=1)

# 填补 BsmtFinType2 缺失值
def fill_BsmtFinType2(row):
    t = all_data.loc[(all_data.OverallCond == row.OverallCond), 'BsmtFinType2'].value_counts()
    return t.index[0]
all_data.loc[all_data.BsmtFinType2.isnull(), 'BsmtFinType2'] = all_data[all_data.BsmtFinType2.isnull()].apply(fill_BsmtFinType2, axis=1)

# 填补 BsmtFinSF1 缺失值
all_data.loc[all_data.BsmtFinSF1.isnull(), 'BsmtFinSF1'] = 550

# 填补 BsmtFinSF2 缺失值
all_data.loc[all_data.BsmtFinSF2.isnull(), 'BsmtFinSF2'] = 0.0

# a = all_data[all_data.BsmtFinSF2.isnull()][['1stFlrSF', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinType1', 'BsmtFinType2']]
# ic(a)
# c = all_data[all_data['BsmtFinType2'] == 'Unf'][['BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', 'BsmtUnfSF']]
# ic(c)
# ic(c.BsmtFinSF2.unique())
# ic(c.BsmtFinSF2.value_counts())

# 填补 TotalBsmtSF 缺失值
"""TotalBsmtSF(地下室总面积) = BsmtFinSF1(地下室装饰面积) + BsmtFinSF2(地下室装饰面积) + BsmtUnfSF(未装饰的地下室面积)"""

all_data.loc[all_data.TotalBsmtSF.isnull(), 'TotalBsmtSF'] = 896.0

# 填补 BsmtUnfSF 缺失值
t = all_data.loc[all_data.BsmtUnfSF.isnull(), 'TotalBsmtSF'] - all_data.loc[all_data.BsmtUnfSF.isnull(), 'BsmtFinSF1'] -  all_data.loc[all_data.BsmtUnfSF.isnull(), 'BsmtFinSF2']
'''获取标量值'''
filled_val = t.values[0]
'''填补'''
all_data.loc[all_data.BsmtUnfSF.isnull(), 'BsmtUnfSF'] = filled_val

# 填补 Electrical 缺失值
all_data.loc[all_data.Electrical.isnull(), 'Electrical'] = 'SBrkr'

# 填补 BsmtFullBath 缺失值
'''第一个缺失值'''
all_data.loc[all_data.Id == 2189, 'BsmtFullBath'] = 0.0
'''第二个缺失值'''
all_data.loc[all_data.Id == 2121, 'BsmtFullBath'] = 1.0

# 填补 BsmtHalfBath 缺失值
'''第一个缺失值'''
all_data.loc[all_data.Id == 2189, 'BsmtHalfBath'] = 0.0
'''第二个缺失值'''
all_data.loc[all_data.Id == 2121, 'BsmtHalfBath'] = 0.0

# 填补 KitchenQual 缺失值
t = all_data.groupby('OverallCond')['KitchenQual'].value_counts().reset_index(name='count')

def fill_kq(row):
    a = t[t.OverallCond == row.OverallCond]['KitchenQual']
    return a.values[0]

all_data.loc[all_data.KitchenQual.isnull(), 'KitchenQual']=all_data[all_data.KitchenQual.isnull()].apply(fill_kq, axis=1)

# 填补 Functional 缺失值
t = all_data.groupby('OverallCond')['Functional'].value_counts().reset_index(name='count')

def fill_kq(row):
    a = t[t.OverallCond == row.OverallCond]['Functional']
    return a.values[0]

all_data.loc[all_data.Functional.isnull(), 'Functional']=all_data[all_data.Functional.isnull()].apply(fill_kq, axis=1)

# 填补 GarageCars 缺失值
t = all_data.groupby(['OverallQual', 'YearBuilt'])['GarageCars'].value_counts().reset_index(name='count')

def fill_gc(row):
    a = t[(t.OverallQual == row.OverallQual) & (t.YearBuilt == row.YearBuilt)]['GarageCars']
    return a.values[0]

all_data.loc[all_data.GarageCars.isnull(), 'GarageCars'] = all_data[all_data.GarageCars.isnull()].apply(fill_gc, axis=1)

# 填补 GarageArea 缺失值
t = all_data.groupby('GarageCars')['GarageArea'].value_counts().reset_index(name='count')

def fill_ga(row):
    a = t[t.GarageCars == row.GarageCars]['GarageArea']
    return a.values[0]

all_data.loc[all_data.GarageArea.isnull(), 'GarageArea']=all_data[all_data.GarageArea.isnull()].apply(fill_ga, axis=1)

# 通过随机森林填补 GarageQual 缺失值
init_f = ['MSSubClass','MSZoning','OverallCond','YearBuilt','YearRemodAdd','Foundation','BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','1stFlrSF','GarageCars','Street','Alley','LotConfig','BldgType','HouseStyle']
X = all_data[all_data.GarageQual.notnull()][init_f]
y = all_data[all_data.GarageQual.notnull()]['GarageQual']
X_dumm = pd.get_dummies(X)
selector = SelectKBest(chi2, k=12)
a = selector.fit(X_dumm, y)

sorted_index = np.argsort(a.scores_)[-10:]

# plt.style.use({'figure.figsize':(30, 14)})
# sns.barplot(x=X_dumm.columns[sorted_index], y=np.sort(np.array(a.scores_))[:], ci=0)
# plt.xticks(rotation=90)
# plt.show()

'''对 GarageQual 影响比较大的列'''
selected_columns = X_dumm.columns[sorted_index]

X = all_data[all_data.GarageQual.notnull()]
X = pd.get_dummies(X)
X = X[selected_columns]
y = all_data[all_data.GarageQual.notnull()]['GarageQual']

X_pred = all_data[all_data.GarageQual.isnull()]
X_pred = pd.get_dummies(X_pred)
X_pred = X_pred[selected_columns]

'''搜索最优参数'''
# pipe = Pipeline([
#     ('rfc', RandomForestClassifier(random_state=1))
# ])
# param_grid = {
#     'rfc__n_estimators': list(range(3, 60, 2)),
#     'rfc__max_depth': list(range(2, 20, 2))
# }
# gsearch = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=10, n_jobs=-1, scoring='accuracy')
# gsearch.fit(X, y)
# ic(gsearch.best_params_, gsearch.best_score_)

'''评估模型'''
pipe = Pipeline([
    ('rfc', RandomForestClassifier(random_state=1, max_depth=6, n_estimators=41))
])
# scores = cross_val_score(estimator=pipe, X=X, y=y, cv=10, n_jobs=-1, scoring='accuracy')
# ic(scores.mean(), np.std(scores))  # 0.944927536231884  0.00354998513446839
pipe.fit(X, y)
y_pred = pipe.predict(X_pred)
all_data.loc[all_data.GarageQual.isnull(), 'GarageQual'] = y_pred

# 通过随机森林填补 GarageCond 缺失值
X = all_data[all_data.GarageCond.notnull()]
X = pd.get_dummies(X)
X = X[selected_columns]
y = all_data[all_data.GarageCond.notnull()]['GarageCond']

X_pred = all_data[all_data.GarageCond.isnull()]
X_pred = pd.get_dummies(X_pred)
X_pred = X_pred[selected_columns]

'''评估模型'''
pipe = Pipeline([
    ('rfc', RandomForestClassifier(random_state=1, max_depth=6, n_estimators=41))
])
# scores = cross_val_score(estimator=pipe, X=X, y=y, cv=10, n_jobs=-1, scoring='accuracy')
# ic(scores.mean(), np.std(scores))  # 0.9612318840579711  0.0023199725497944725
pipe.fit(X, y)
y_pred = pipe.predict(X_pred)
all_data.loc[all_data.GarageCond.isnull(), 'GarageCond'] = y_pred

# 填补 GarageFinish 缺失值
g = all_data.groupby('GarageCond')['GarageFinish'].value_counts().reset_index(name='count')

def fill_gf(row):
    a = g[g.GarageCond == row.GarageCond]['GarageFinish']
    return a.values[0]

all_data.loc[all_data.GarageFinish.isnull(), 'GarageFinish'] = all_data[all_data.GarageFinish.isnull()].apply(fill_gf, axis=1)

# 填补 GarageYrBlt 缺失值
'''将异常值设为nan'''
all_data.loc[all_data.GarageYrBlt >= 2020, 'GarageYrBlt'] = np.nan

'''第一次填补'''
my_df = all_data.groupby('YearBuilt')['GarageYrBlt'].value_counts().reset_index(name='count')

def fill_gyb(row):
    w = my_df[my_df.YearBuilt == row.YearBuilt]['GarageYrBlt']
    if len(w.values) == 0:
        return np.nan
    return w.values[0]

all_data.loc[all_data.GarageYrBlt.isnull(), 'GarageYrBlt'] = all_data[all_data.GarageYrBlt.isnull()].apply(fill_gyb, axis=1)

'''第二次填补'''
my_df2 = all_data.groupby('YearRemodAdd')['GarageYrBlt'].value_counts().reset_index(name='count')

def fill_gyb2(row):
    w2 = my_df2[my_df2.YearRemodAdd == row.YearRemodAdd]['GarageYrBlt']
    return w2.values[0]

all_data.loc[all_data.GarageYrBlt.isnull(), 'GarageYrBlt'] = all_data[all_data.GarageYrBlt.isnull()].apply(fill_gyb2, axis=1)

# 填补 GarageType 缺失值
init_f = ['MSSubClass','MSZoning','OverallCond','YearBuilt','YearRemodAdd','Foundation','BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','1stFlrSF','GarageCars','Street','Alley','LotConfig','BldgType','HouseStyle']

X = all_data[all_data.GarageType.notnull()][init_f]
y = all_data[all_data.GarageType.notnull()]['GarageType']

X_dumm = pd.get_dummies(X)
selector = SelectKBest(chi2)

a = selector.fit(X_dumm, y)
sorted_index = np.argsort(a.scores_)[-13:]

'''对 GarageType 影响比较大的列'''
selected_columns = X_dumm.columns[sorted_index]

X_gt = all_data[all_data.GarageType.notnull()]
X_gt = pd.get_dummies(X_gt)
X_gt = X_gt[selected_columns]
y_gt = all_data[all_data.GarageType.notnull()]['GarageType']

X_gt_test = all_data[all_data.GarageType.isnull()]
X_gt_test = pd.get_dummies(X_gt_test)
X_gt_test = X_gt_test[selected_columns]


'''搜索最优参数'''
# pipe = Pipeline([
#     ('rfc', RandomForestClassifier(random_state=3))
# ])
# param_grid = {
#     'rfc__n_estimators': list(range(3, 60, 2)),
#     'rfc__max_depth': list(range(2, 20, 2)),
#     'rfc__max_features': ['auto', 'sqrt', 'log2']
# }
# gsearch = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=10, n_jobs=-1, scoring='accuracy', verbose=2)
# gsearch.fit(X_gt, y_gt)
# ic(gsearch.best_params_, gsearch.best_score_)

'''评估模型'''
pipe_best = Pipeline([
    ('rfc', RandomForestClassifier(random_state=3, max_depth=10, n_estimators=33))
])
# scores = cross_val_score(estimator=pipe_best, X=X_gt, y=y_gt, cv=10, n_jobs=-1, scoring='accuracy')
# ic(np.mean(scores), np.std(scores))

pipe_best.fit(X_gt, y_gt)
y_gt_pred = pipe_best.predict(X_gt_test)
all_data.loc[all_data.GarageType.isnull(), 'GarageType'] = y_gt_pred

# 填补 SaleType 缺失值
g = all_data.groupby('SaleCondition')['SaleType'].value_counts().reset_index(name='count')

def fill_st(row):
    a = g[g.SaleCondition == row.SaleCondition]['SaleType']
    return a.values[0]

all_data.loc[all_data.SaleType.isnull(), 'SaleType'] = all_data[all_data.SaleType.isnull()].apply(fill_st, axis=1)

# 填补 FireplaceQu 缺失值
'''第一次'''
g = all_data.groupby('Heating')['FireplaceQu'].value_counts().reset_index(name='count')

def fill_fq(row):
    a = g[g.Heating == row.Heating]['FireplaceQu']
    return a.values[0]

all_data.loc[(all_data.FireplaceQu.isnull()) & (all_data.Heating.isin(['GasA', 'OthW', 'Grav'])), 'FireplaceQu'] = all_data[(all_data.FireplaceQu.isnull()) & (all_data.Heating.isin(['GasA', 'OthW', 'Grav']))].apply(fill_fq, axis=1)

'''第二次'''
g2 = all_data.groupby('HeatingQC')['FireplaceQu'].value_counts().reset_index(name='count')

def fill_fq2(row):
    a = g2[g2.HeatingQC == row.HeatingQC]['FireplaceQu']
    return a.values[0]

all_data.loc[(all_data.FireplaceQu.isnull()), 'FireplaceQu']=all_data[(all_data.FireplaceQu.isnull())].apply(fill_fq2, axis=1)

# 增加新特征 HasPool
def add_HasPool(row):
    if row.PoolArea == 0:
        return 0
    else:
        return 1
all_data['HasPool'] = all_data.apply(add_HasPool, axis=1)

# 填补Fence
'''第一次填补'''
g = all_data.groupby(by=['HasPool','OverallCond','Street'])['Fence'].value_counts().reset_index(name='count')
def fill_Fence(row):
    a = g[(g.HasPool==row.HasPool) & (g.OverallCond==row.OverallCond) & (g.Street==row.Street)]['Fence']
    if len(a.values) == 0:
        return np.nan
    else:
        return a.values[0]
all_data.loc[all_data.Fence.isnull(), 'Fence']=all_data[all_data.Fence.isnull()].apply(fill_Fence, axis=1)

'''第二次填补'''
g = all_data.groupby(by=['HasPool','OverallCond','MSZoning'])['Fence'].value_counts().reset_index(name='count')
def fill_Fence2(row):
    a = g[(g.HasPool==row.HasPool) & (g.OverallCond==row.OverallCond) & (g.MSZoning==row.MSZoning)]['Fence']
    return a.values[0]
all_data.loc[all_data.Fence.isnull(), 'Fence'] = all_data[all_data.Fence.isnull()].apply(fill_Fence2, axis=1)

# 处理 WoodDeckSF 异常值
all_data.loc[all_data.WoodDeckSF == 1424, 'WoodDeckSF'] = 100

# # 将 PoolArea 变成离散型变量
# def transform_pa(row):
#     if row.PoolArea >= 0 and row.PoolArea <= 480:
#         return 0
#     elif row.PoolArea > 578 and row.PoolArea <= 650:
#         return 0
#     else:
#         return 1
# all_data['PoolArea2'] = all_data.apply(transform_pa, axis=1)


all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
all_data['TotalPorch'] = all_data['OpenPorchSF'] + all_data['EnclosedPorch'] + all_data['3SsnPorch'] + all_data['ScreenPorch']
all_data['OverallAssess'] = all_data['OverallQual'] + all_data['OverallCond']

################################################### 特征工程完毕

In [18]:
all_data.GarageQual.value_counts()

TA    2758
Fa     127
Gd      24
Po       5
Ex       3
Name: GarageQual, dtype: int64

In [19]:
all_data.GarageCond.value_counts()

TA    2811
Fa      74
Gd      15
Po      14
Ex       3
Name: GarageCond, dtype: int64

In [5]:
d = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1
}
all_data['ExterQual2'] = all_data['ExterQual'].map(d)

In [6]:
all_data.ExterQual2.value_counts()

3    1798
4     979
5     105
2      35
Name: ExterQual2, dtype: int64